In [ ]:
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, VBox, HBox

class RecipeRecommender:
    def __init__(self, data_url):
        # Load dataset
        self.df = pd.read_csv(data_url)
        self.df = self.df.head(100)
        
        self.user_favorites = []
        self._prepare_data()
        
        # Create the UI components
        self.dish_dropdown = widgets.Dropdown(
            options=[(name, idx) for idx, name in enumerate(self.df['name'])],
            disabled=False,
            layout=widgets.Layout(width='250px')
        )
        
        self.select_button = widgets.Button(description="Add to Favorites", layout=widgets.Layout(width='150px'))
        self.select_button.on_click(self.select_recipes)

        self.output_recs = widgets.Output(layout=widgets.Layout(width='600px', height='400px', overflow='auto'))
        self.favorites_output = widgets.Output(layout=widgets.Layout(width='250px', height='300px', overflow='auto'))
        
        self.select_button.on_click(self.refresh_recommendations)

        header = HTML("<h1>RECSYS</h1>")
        
        left_panel = VBox([
            self.dish_dropdown,
            self.select_button,
            self.favorites_output
        ], layout=widgets.Layout(width='300px', padding='10px'))

        right_panel = VBox([
            self.output_recs
        ], layout=widgets.Layout(width='600px', padding='10px'))

        self.main_layout = HBox([left_panel, right_panel], layout=widgets.Layout(align_items='center', justify_content='center'))

        # Initialize the display
        display(header)
        display(self.main_layout)
        self.refresh_recommendations()

    def _prepare_data(self):
        def extract_ingredients(ingredient_string):
            matches = re.findall(r"'([^']+)'(?=\s*:\s*)", ingredient_string)
            return matches

        self.df['ingredients_extracted'] = self.df['ingredients'].apply(extract_ingredients)
        self.df['ingredients_str'] = self.df['ingredients_extracted'].apply(lambda x: ' '.join(x))

        count_vectorizer = CountVectorizer()
        self.ingredient_matrix = count_vectorizer.fit_transform(self.df['ingredients_str'])

    def refresh_recommendations(self, button=None):
        recommended_recipes = self.recommend_based_on_favorites()
        with self.output_recs:
            clear_output(wait=True)
            if recommended_recipes.empty:
                display(HTML("<p>No recommendations yet. Add some favorites!</p>"))
            else:
                display(recommended_recipes.style.set_table_attributes('class="recommend-table"'))

    def recommend_based_on_favorites(self, n_recommendations=15):
        indices = self.df.index[self.df['name'].isin(self.user_favorites)].tolist()
        if not indices:
            return pd.DataFrame()
        user_vectors = self.ingredient_matrix[indices]
        sim_scores = cosine_similarity(user_vectors, self.ingredient_matrix).mean(axis=0)
        similar_recipes = pd.Series(sim_scores, index=self.df.index).sort_values(ascending=False)
        recommendations = similar_recipes[~similar_recipes.index.isin(indices)].head(n_recommendations)
        return self.df.loc[recommendations.index, ['name', 'url']]

    def select_recipes(self, button=None):
        selected_index = self.dish_dropdown.value
        selected_dish = self.df.iloc[selected_index]['name']
        if selected_dish not in self.user_favorites:
            self.user_favorites.append(selected_dish)
            self.update_favorites_display()

    def update_favorites_display(self):
        self.favorites_output.clear_output()
        with self.favorites_output:
            for dish in self.user_favorites:
                display(HTML(f'<div class="favorite-item">{dish}</div>'))

    @classmethod
    def from_url(cls, url):
        return cls(url)

# Use this line to instantiate the app with your data
DATA_URL = "povarenok_recipes_2021_06_16.csv"  # Update this to your desired URL
app = RecipeRecommender.from_url(DATA_URL)